In [79]:
import pandas as pd
import numpy as np

In [ ]:
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

shared_links = [
    "https://drive.google.com/file/d/1wXdR-snwm65h6EVpScC7TChXlHmcyx6G/view?usp=drive_link", 
    "https://drive.google.com/file/d/1AddWngzGBR-8u7U3X4s78WIOusOIbubj/view?usp=drive_link", 
    "https://drive.google.com/file/d/1ta7thsqQXqPBBjA256jL5VoClicnhDvE/view?usp=drive_link", 
    "https://drive.google.com/file/d/1PUOs8DpkJ2q5fmAttONWeVImgFcaomFh/view?usp=drive_link", 
    "https://drive.google.com/file/d/1mAHueitZdsP8U0ZJTaVvfy4bbjHJEmyo/view?usp=drive_link", 
    "https://drive.google.com/file/d/16dxRfYiPVaEFoL6sk1-kEw9GBnVpo7ZH/view?usp=drive_link",
    "https://drive.google.com/file/d/1UdQPw2EhLzzmil9iRFPpEkb34EiSvrCK/view?usp=drive_link",
    "https://drive.google.com/file/d/1jwPOOtLFhouST6HUIuDaYUuLQA1EHCmg/view?usp=drive_link",
    "https://drive.google.com/file/d/1Na-dW5nY0owCB4bubFtKu145eu0lb-Rw/view?usp=drive_link
]

dfs = []
for year, link in zip(range(2015, 2024), shared_links):
    direct_link = get_direct_link(link)
    df = pd.read_csv(direct_link, encoding='utf-8')
    df["Year"] = year
    
    dfs.append(df)

combined_df = pd.concat(dfs)

combined_df['LastName_W'] = combined_df['Winner'].str.extract(r'(.*)\s\w\.$')
combined_df['LastName_L'] = combined_df['Loser'].str.extract(r'(.*)\s\w\.$')

combined_df['LastName_W'] = combined_df['LastName_W'].str.strip()
combined_df['LastName_L'] = combined_df['LastName_L'].str.strip()

In [81]:
dfs = []
columns = []
for year in range(2015,2024):
    file_path = data_dir + f"{year}.csv"
    df = pd.read_csv(file_path, encoding='utf-8')
    df["Year"] = int(year)
    
    dfs.append(df)
    columns.append(df.columns.tolist())

In [82]:
combined_df = pd.concat(dfs, ignore_index=True)

In [83]:
columns_choice = ['Date', 'Year', 'Winner', 'Loser', 'Surface', 'Round', 'Series', 'B365W', 'B365L', 'AvgW', 'AvgL', 'MaxW', 'MaxL', 'WRank', 'LRank']
columns_to_drop = [col for col in combined_df.columns if col not in columns_choice]
combined_df = combined_df.drop(columns=columns_to_drop)
combined_df.reset_index(inplace=True)
combined_df.to_csv('unclean_mens_tennis_odds.csv', index=False)

In [85]:
disagree_rows = combined_df[((combined_df['B365W'] < combined_df['B365L']) & 
                            (combined_df['AvgW'] > combined_df['AvgL'])) | 
                           ((combined_df['B365W'] > combined_df['B365L']) & 
                            (combined_df['AvgW'] < combined_df['AvgL']))]
significant_disagree_rows = disagree_rows[abs(disagree_rows['B365W'] - disagree_rows['B365L']) > 0.2]
significant_disagree_rows

,index,Date,Series,Surface,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year
4274,4274,2016-07-10,Grand Slam,Grass,The Final,Murray A.,Raonic M.,2.0,7.0,1.25,4.330,4.36,1.30,3.97,1.26,2016
7543,7543,2017-09-26,ATP250,Hard,1st Round,Copil M.,Jaziri M.,83.0,80.0,2.00,1.720,2.00,2.10,1.89,1.90,2017
7603,7603,2017-10-02,ATP500,Hard,1st Round,Uchiyama Y.,Skugor F.,214.0,324.0,2.00,1.720,2.00,2.00,1.89,1.90,2017
9773,9773,2018-07-31,ATP500,Hard,1st Round,Kudla D.,Lacko L.,85.0,75.0,1.72,2.000,2.00,2.05,1.93,1.87,2018
10997,10997,2019-02-19,ATP250,Hard,1st Round,Berrettini M.,Chardy J.,48.0,35.0,2.00,1.720,2.06,2.00,1.90,1.91,2019
13312,13312,2020-01-23,Grand Slam,Hard,2nd Round,Nadal R.,Delbonis F.,1.0,76.0,29.00,0.967,1.02,35.00,1.01,21.52,2020
13948,13948,2020-09-20,Masters 1000,Clay,Semifinals,Djokovic N.,Ruud C.,1.0,34.0,1.16,5.000,1.20,5.70,17.00,5.05,2020
14071,14071,2020-10-01,Grand Slam,Clay,2nd Round,Djokovic N.,Berankis R.,1.0,66.0,34.00,0.971,1.02,41.00,1.01,21.54,2020
14541,14541,2021-02-02,ATP250,Hard,2nd Round,Gerasimov E.,Fucsovics M.,78.0,55.0,2.50,1.500,2.60,1.60,1.46,1.54,2021
14641,14641,2021-02-10,Grand Slam,Hard,2nd Round,Djokovic N.,Tiafoe F.,1.0,64.0,29.00,0.967,1.02,29.00,1.01,19.55,2021


In [86]:
def swap_odds(row_index, bookmakers, dataframe):
    for bookmaker in bookmakers:
        w_column = f"{bookmaker}W"
        l_column = f"{bookmaker}L"
        
        # Swap the values
        dataframe.at[row_index, w_column], dataframe.at[row_index, l_column] = dataframe.at[row_index, l_column], dataframe.at[row_index, w_column]
    
    return dataframe

In [87]:
combined_df = swap_odds(4274, ['Avg', 'Max'], combined_df)
combined_df = swap_odds(13312, ['B365'], combined_df)
combined_df = swap_odds(14071, ['B365'], combined_df)
combined_df = swap_odds(14641, ['B365'], combined_df)
combined_df = swap_odds(15607, ['B365'], combined_df)
combined_df = swap_odds(15652, ['B365'], combined_df)

In [88]:
disagree_rows = combined_df[((combined_df['B365W'] < combined_df['B365L']) & 
                            (combined_df['AvgW'] > combined_df['AvgL'])) | 
                           ((combined_df['B365W'] > combined_df['B365L']) & 
                            (combined_df['AvgW'] < combined_df['AvgL']))]
significant_disagree_rows = disagree_rows[abs(disagree_rows['B365W'] - disagree_rows['B365L']) > 0.2]
significant_disagree_rows

,index,Date,Series,Surface,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year
7543,7543,2017-09-26,ATP250,Hard,1st Round,Copil M.,Jaziri M.,83.0,80.0,2.00,1.72,2.00,2.10,1.89,1.90,2017
7603,7603,2017-10-02,ATP500,Hard,1st Round,Uchiyama Y.,Skugor F.,214.0,324.0,2.00,1.72,2.00,2.00,1.89,1.90,2017
9773,9773,2018-07-31,ATP500,Hard,1st Round,Kudla D.,Lacko L.,85.0,75.0,1.72,2.00,2.00,2.05,1.93,1.87,2018
10997,10997,2019-02-19,ATP250,Hard,1st Round,Berrettini M.,Chardy J.,48.0,35.0,2.00,1.72,2.06,2.00,1.90,1.91,2019
13948,13948,2020-09-20,Masters 1000,Clay,Semifinals,Djokovic N.,Ruud C.,1.0,34.0,1.16,5.00,1.20,5.70,17.00,5.05,2020
14541,14541,2021-02-02,ATP250,Hard,2nd Round,Gerasimov E.,Fucsovics M.,78.0,55.0,2.50,1.50,2.60,1.60,1.46,1.54,2021
15568,15568,2021-05-31,Grand Slam,Clay,1st Round,Mager G.,Gojowczyk P.,87.0,134.0,2.62,1.50,1.48,8.00,1.28,5.34,2021
17706,17706,2022-04-10,ATP250,Clay,The Final,Opelka R.,Isner J.,18.0,27.0,1.61,2.30,1.68,2.53,161.00,2.31,2022
19799,19799,2023-02-08,ATP250,Hard,1st Round,Gomez E.,Diallo G.,102.0,211.0,1.73,2.10,2.00,1.80,2.00,1.80,2023


In [89]:
idx_drop = [13948, 14541, 17706, 15568]
combined_df = combined_df.drop(idx_drop)

In [90]:
def filter_invalid_probabilities(df, bookmakers):
    odds1_col = f'{bookmakers}W'
    odds2_col = f'{bookmakers}L'
    
    df['prob1'] = 1 / df[odds1_col]
    df['prob2'] = 1 / df[odds2_col]

    invalid_rows = df[(df['prob1'] + df['prob2'] < 1) | (df['prob1'] + df['prob2'] > 1.1)]

    return invalid_rows

In [91]:
invalid_df = filter_invalid_probabilities(combined_df, 'Avg')
invalid_df

,index,Date,Series,Surface,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year,prob1,prob2
918,918,2015-04-21,ATP500,Clay,1st Round,Sousa J.,De Schepper K.,56.0,137.0,NaN,NaN,1.37,2.65,1.37,2.63,2015,0.729927,0.380228
4795,4795,2016-09-02,Grand Slam,Hard,3rd Round,Tsonga J.W.,Anderson K.,11.0,35.0,1.44,2.75,1.48,3.05,1.42,3.82,2016,0.704225,0.261780
5781,5781,2017-02-28,ATP500,Hard,1st Round,Thompson J.,Lopez F.,79.0,31.0,2.37,1.53,2.60,2.39,1.63,1.56,2017,0.613497,0.641026
9243,9243,2018-06-07,Grand Slam,Clay,Quarterfinals,Del Potro J.M.,Cilic M.,6.0,4.0,1.50,2.62,1.55,2.75,1.50,1.65,2018,0.666667,0.606061
9839,9839,2018-08-08,Masters 1000,Hard,2nd Round,Haase R.,Youzhny M.,39.0,104.0,1.25,3.75,1.44,11.63,1.32,4.21,2018,0.757576,0.237530


In [92]:
invalid_df = filter_invalid_probabilities(combined_df, 'B365')
invalid_df

,index,Date,Series,Surface,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year,prob1,prob2
10516,10516,2018-11-13,Masters Cup,Hard,Round Robin,Federer R.,Thiem D.,3.0,8.0,1.30,3.00,1.40,3.65,1.35,3.28,2018,0.769231,0.333333
16222,16222,2021-08-11,Masters 1000,Hard,2nd Round,Tsitsipas S.,Humbert U.,3.0,26.0,1.36,2.50,1.36,3.55,1.33,3.24,2021,0.735294,0.400000
16234,16234,2021-08-12,Masters 1000,Hard,2nd Round,Tiafoe F.,Shapovalov D.,52.0,10.0,3.20,2.50,3.56,1.40,3.25,1.33,2021,0.312500,0.400000
16236,16236,2021-08-12,Masters 1000,Hard,3rd Round,Tsitsipas S.,Khachanov K.,3.0,29.0,1.36,2.50,1.44,3.40,1.36,3.10,2021,0.735294,0.400000
16244,16244,2021-08-13,Masters 1000,Hard,Quarterfinals,Tsitsipas S.,Ruud C.,3.0,12.0,1.25,2.50,1.29,4.20,1.25,3.91,2021,0.800000,0.400000
17120,17120,2022-01-25,Grand Slam,Hard,Quarterfinals,Nadal R.,Shapovalov D.,5.0,14.0,1.33,2.62,1.40,3.68,1.34,3.25,2022,0.751880,0.381679


In [93]:
idx_drop = [5781, 9243, 4795, 16234]
combined_df = combined_df.drop(idx_drop)

In [94]:
combined_df.reset_index(inplace=True)

In [95]:
combined_df.iloc[combined_df.B365W.nsmallest(15).index]

,level_0,index,Date,Series,Surface,Round,Winner,Loser,WRank,LRank,B365W,B365L,MaxW,MaxL,AvgW,AvgL,Year,prob1,prob2
13309,13312,13312,2020-01-23,Grand Slam,Hard,2nd Round,Nadal R.,Delbonis F.,1.0,76.0,0.967,29.0,1.02,35.00,1.01,21.52,2020,1.034126,0.034483
14636,14641,14641,2021-02-10,Grand Slam,Hard,2nd Round,Djokovic N.,Tiafoe F.,1.0,64.0,0.967,29.0,1.02,29.00,1.01,19.55,2021,1.034126,0.034483
15601,15607,15607,2021-06-01,Grand Slam,Clay,1st Round,Djokovic N.,Sandgren T.,1.0,66.0,0.967,29.0,1.02,35.00,1.01,23.01,2021,1.034126,0.034483
15646,15652,15652,2021-06-05,Grand Slam,Clay,3rd Round,Djokovic N.,Berankis R.,1.0,93.0,0.967,29.0,1.01,34.00,1.01,21.85,2021,1.034126,0.034483
14013,14017,14017,2020-09-28,Grand Slam,Clay,1st Round,Nadal R.,Gerasimov E.,2.0,83.0,0.971,34.0,1.02,35.00,1.01,22.68,2020,1.029866,0.029412
14067,14071,14071,2020-10-01,Grand Slam,Clay,2nd Round,Djokovic N.,Berankis R.,1.0,66.0,0.971,34.0,1.02,41.00,1.01,21.54,2020,1.029866,0.029412
63,63,63,2015-01-06,ATP250,Hard,1st Round,Stakhovsky S.,Ali Mutawa J.M.,57.0,NaN,1.000,101.0,1.02,101.00,1.01,24.01,2015,1.000000,0.009901
1283,1283,1283,2015-05-26,Grand Slam,Clay,1st Round,Nadal R.,Halys Q.,7.0,296.0,1.000,51.0,1.01,61.00,1.01,27.09,2015,1.000000,0.019608
2095,2095,2095,2015-08-31,Grand Slam,Hard,1st Round,Djokovic N.,Souza J.,1.0,91.0,1.000,67.0,1.01,91.00,1.01,26.17,2015,1.000000,0.014925
2685,2685,2685,2016-01-04,ATP250,Hard,1st Round,Chardy J.,Zayid M. S.,31.0,1512.0,1.000,51.0,1.01,51.00,1.01,26.32,2016,1.000000,0.019608


In [96]:
combined_df.loc[combined_df['B365W'] < 1.01, 'B365W'] = 1.01

In [97]:
combined_df['AvgW_prob'] = 1 / combined_df['AvgW']
combined_df['AvgL_prob'] = 1 / combined_df['AvgL']
combined_df['B365W_prob'] = 1 / combined_df['B365W']
combined_df['B365L_prob'] = 1 / combined_df['B365L']

combined_df['W_diff'] = abs(combined_df['AvgW_prob'] - combined_df['B365W_prob'])
combined_df['L_diff'] = abs(combined_df['AvgL_prob'] - combined_df['B365L_prob'])

In [98]:
Q1 = combined_df['W_diff'].quantile(0.25)
Q3 = combined_df['W_diff'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = combined_df[(combined_df['W_diff'] >= lower_bound) & (combined_df['W_diff'] <= upper_bound)]
outliers_removed = combined_df.shape[0] - filtered_data.shape[0]
print(outliers_removed)
combined_df = filtered_data

721


In [99]:
Q1 = combined_df['L_diff'].quantile(0.25)
Q3 = combined_df['L_diff'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_data = combined_df[(combined_df['L_diff'] >= lower_bound) & (combined_df['L_diff'] <= upper_bound)]
outliers_removed = combined_df.shape[0] - filtered_data.shape[0]
print(outliers_removed)
combined_df = filtered_data

591


In [100]:
columns_choice = ['Date', 'Year', 'Winner', 'Loser', 'Surface', 'Round', 'Series', 'B365W', 'B365L', 'AvgW', 'AvgL', 'MaxW', 'MaxL', 'WRank', 'LRank']
columns_to_drop = [col for col in combined_df.columns if col not in columns_choice]
combined_df = combined_df.drop(columns=columns_to_drop)
combined_df.reset_index(inplace=True)

In [102]:
combined_df.to_csv('mens_tennis_odds.csv', index=False)